In [198]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import folium
routes_normal_with_R23 = pd.read_csv("data/routes_with_23.csv")
custom3 = pd.read_csv("data/custom/custom3.csv")
custom11 = pd.read_csv("data/custom/custom11.csv")
custom13 = pd.read_csv("data/custom/custom13.csv")
custom23A = pd.read_csv("data/custom/custom23A.csv")
custom23B = pd.read_csv("data/custom/custom23B.csv")
shuttle = pd.read_csv("data/custom/shuttle.csv")
full_routes  = pd.concat([routes_normal_with_R23, custom3,custom11, custom13, custom23A, custom23B, shuttle])
full_routes.reset_index(drop=True, inplace=True)
full_routes['route_short_name'] = full_routes['route_short_name'].astype(str)

full_routes

C:\Users\marti\AppData\Local\Temp\ipykernel_4300\240250908.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  routes_normal_with_R23 = pd.read_csv("data/routes_with_23.csv")


,route_short_name,direction_id,shape_pt_lat,shape_pt_lon,shape_dist_traveled,route_long_name
0,101,1,63.430569,10.391581,0.0,Nattbuss Byåsen/Flatåsen
1,101,1,63.430569,10.391726,7.0,Nattbuss Byåsen/Flatåsen
2,101,1,63.430604,10.391839,14.0,Nattbuss Byåsen/Flatåsen
3,101,1,63.430594,10.392154,30.0,Nattbuss Byåsen/Flatåsen
4,101,1,63.430471,10.392243,44.0,Nattbuss Byåsen/Flatåsen
...,...,...,...,...,...,...
289346,VM2025,0,63.430471,10.392243,7254.0,Shuttlebuss VM
289347,VM2025,0,63.430594,10.392154,7268.0,Shuttlebuss VM
289348,VM2025,0,63.431945,10.391923,7268.0,Shuttlebuss VM
289349,VM2025,0,63.431958,10.394273,7268.0,Shuttlebuss VM


In [220]:
from folium import DivIcon
def busseluringene(data, color_mapping):
    m = folium.Map(location=[63.4305, 10.3951], zoom_start=12)
    route_short_names = color_mapping.keys()
    route_direction = {"from","to"}
    route_groups = {}
    #dir_groups = {}
    
    for route_short_name in route_short_names:
        #dir_groups[direction] = folium.FeatureGroup(name=direction)
        for direction in route_direction:
            if not route_short_name in data["route_short_name"].values: continue
            route_name_full = f"{route_short_name}, {direction}"
            route_group = folium.FeatureGroup(name=route_name_full,show=False)
            route_groups[route_name_full] = route_group

            route = data[data["route_short_name"] == route_short_name]
            route = route[route["direction_id"] == color_mapping[route_short_name]['direction'][direction]]
            lastIndex = route.index[-1]

            if color_mapping is not None:
                color = color_mapping[route_short_name]['color']  # Accessing color from the new format
            else: color = "black"
            
            for index, row in route.iterrows():
                # Get the latitude and longitude of each point
                lat = row['shape_pt_lat']
                lon = row['shape_pt_lon']
                # Create a polyline between the current point and the next point with the assigned color
                if index < lastIndex - 1:
                    next_lat = route.loc[index + 1, 'shape_pt_lat']
                    next_lon = route.loc[index + 1, 'shape_pt_lon']
                    points = [(lat, lon), (next_lat, next_lon)]
                    
                    line = folium.PolyLine(points, color=color)
                    line.add_to(route_groups[route_name_full])
                    #line.add_to(dir_groups[direction])
            #TODO: How to add markers that do not scale to the map such as they can be used to show the route number, and potensial bussstops
            #plugins.PolyLineTextPath(line, f"Route {route_short_name}", offset=20).add_to(route_group)

    for route_group in route_groups.values():
        route_group.add_to(m)
    folium.LayerControl().add_to(m)
    

    
    return m




In [224]:
from nbconvert import HTMLExporter
import codecs
color_mapping = {
    "2": {'color': 'blue', 'direction': {'from': 0, 'to': 1}},
    "3": {'color': 'green', 'direction': {'from': 0, 'to': 1}},
    "VM3": {'color': 'green', 'direction': {'from': 0, 'to': 1}},  
    "11": {'color': 'orange', 'direction': {'from': 0, 'to': 1}},
    "VM11": {'color': 'orange', 'direction': {'from': 0, 'to': 1}}, 
    "13": {'color': 'purple', 'direction': {'from': 0, 'to': 1}},
    "VM13": {'color': 'purple', 'direction': {'from': 0, 'to': 1}}, 
    "23": {'color': 'brown', 'direction': {'from': 0, 'to': 1}},
    "VM23A": {'color': 'brown', 'direction': {'from': 0, 'to': 1}},
    "VM23B": {'color': 'brown', 'direction': {'from': 1, 'to': 0}},
    "VM2025": {'color': 'black', 'direction': {'from': 0, 'to': 1}}
    "1": {'color': 'red', 'direction': {'from': 0, 'to': 1}},
    "40": {'color': 'pink', 'direction': {'from': 0, 'to': 1}},

}

busseluringene(full_routes, color_mapping).save("index.html")


## Used to make routes
This function is use to make routes. It plots all the geolocations with their coordinates such that the right points easliy could be found.

In [27]:
def plot_all(data):
    m = folium.Map(location=[63.4305, 10.3951], zoom_start=12)
    route_short_names = color_mapping.keys()

    lastIndex = data.index[-1]
    prev_lat = None
    prev_lon = None
    for index, row in data.iterrows():
        # Get the latitude and longitude of each point
        lat = row['shape_pt_lat']
        lon = row['shape_pt_lon']
        
        # Create a marker with information on latitude and longitude
        folium.Marker(location=[lat, lon], popup=f"Lat: {lat}, Lon: {lon}").add_to(m)
        
        # Create a polyline between the previous point and the current point with the assigned color
        if prev_lat is not None and prev_lon is not None:
            points = [(prev_lat, prev_lon), (lat, lon)]
            folium.PolyLine(points, color="red").add_to(m)
        
        prev_lat = lat
        prev_lon = lon
            
    return m

In [123]:
x = pd.read_csv("data/custom/shuttle.csv")
plot_all(x)    